In [1]:
import pysentiment as ps
import pandas as pd
import numpy as np
import keras
from keras.preprocessing import text, sequence
import sys
sys.path.insert(0,'..')

from models import models
from evaluation import evaluation
from preprocess import preprocess

Using TensorFlow backend.
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/users/kostadin

In [6]:
def make_df(train_path, test_path, max_features, maxlen, list_classes, word_index):
    train = pd.read_csv(train_path,names=['id','sent','a','text'],sep='\t')
    test = pd.read_csv(test_path,names=['id','sent','a','text'],sep='\t')

    y = train['sent'].values
    y_test = test['sent'].values
    
    list_sentences_train = train["text"].values
    list_sentences_test = test["text"].values

    tokenizer = text.Tokenizer(num_words=max_features)
    tokenizer.word_index = word_index
    list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
    list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
    X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

    return X_t, X_te, y, y_test

def create_sequence(word_index, sent, maxlen):
    token = text.Tokenizer()
    token.word_index=word_index
    tokenized_text = token.texts_to_sequences(sent)
    X_text = sequence.pad_sequences(tokenized_text, maxlen=maxlen)
    return X_text

def create_embedding_matrix(word2vec,word_index):
    nb_words = max_features
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if word in word2vec.vocab:
            embedding_matrix[i] = word2vec.word_vec(word)
    return embedding_matrix

def make_word2vec(model,max_features,embed_size):
  word_list=dict();
  for idx,k in enumerate(model.vocab.keys()):
    if (idx<max_features):
      word_list[k]=idx;
  embedding_matrix = create_embedding_matrix(model,word_list)
  return embedding_matrix,word_list

In [7]:
from gensim.models import KeyedVectors
# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('/home/users/kostadin.mishev/datasets/word2vec/word2vec-GoogleNews-vectors/GoogleNews-vectors-negative300.bin', binary=True)

MemoryError: Unable to allocate 3.35 GiB for an array with shape (3000000, 300) and data type float32

In [4]:
max_features = 19200
maxlen = 64
embed_size = 300
list_classes = ["sent"]
embedding_vector, word_list = make_word2vec(model,embed_size=300,max_features=max_features)

In [8]:
xtr, xte, y, y_test= make_df(r"/home/users/kostadin.mishev/phd/dataset/train/train.tsv",
                                  r"/home/users/kostadin.mishev/phd/dataset/dev/dev.tsv",
                                  max_features, maxlen, list_classes, word_list)
y = keras.utils.to_categorical(y, 2)

In [9]:
xtr[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,   306,   422, 10147,    14,   295,   213,
        1859], dtype=int32)

In [11]:
embedding_vector.shape

(19200, 300)

In [ ]:
rnn = models.BiGRUAttClassifier(64,input_shape=(maxlen, ),useEmb=True,maxlen=64,embedding_matrix=embedding_vector,max_features=max_features, embed_size=300)
rnn.fit(xtr, y)